In [1]:
import os                       # 운영 체제(OS)와 상호작용할 수 있게 해주는 내장 모듈
import pandas as pd
import torch
import RNNutils_aran as utils 

In [2]:
# 폴더 이름 리스트 빼오기
DATA_PATH='../data/NLP_Dataset/'
file_list=os.listdir(DATA_PATH)[:-1]
print(file_list)

# 정치(0), 경제(1), 사회(2), 생활/문화(3), 세계(4), 기술/IT(5), 연예(6), 스포츠(7)

['0', '1', '2', '3', '4', '5', '6', '7']


In [3]:
# 라벨과
texts=[]
labels=[]

# file_list에 있는 각 label에 대해 반복
for label in file_list:
    # label 디렉토리 내의 파일 목록을 가져옴
    text_list = os.listdir(f'{DATA_PATH}/{label}')
    
    # text_list에 있는 각 파일에 대해 반복
    for tt in text_list:
        # label 디렉토리 내의 각 파일을 읽기 모드로 열고, f라는 파일 객체로 참조
        with open(f'{DATA_PATH}/{label}/{tt}') as f:
            # 파일의 내용을 문자열로 읽어들임
            a = f.read()
            # 읽은 텍스트를 texts 리스트에 추가
            texts.append(a)
            # label을 정수로 변환하여 labels 리스트에 추가
            labels.append(int(label))


In [4]:
print(f'texts number : {len(texts)}')
print(f'labels number : {len(labels)}')

texts number : 1600
labels number : 1600


In [5]:
# 빈 데이터 프레임을 만들고 거기안에 각 text와 label추가 

data=pd.DataFrame()
data['text']=texts
data['label']=labels
data

,text,label
0,"동남아 담당' 北 최희철 부상 베이징 도착…싱가포르행 주목\t최 부상, 행선지·방문...",0
1,"예결위, 추경 막바지 심사 진통…여야 충돌\t(서울=연합뉴스) 김남권 기자 = 국회...",0
2,외압 논란·항명 사태…산 넘고 물 건넌 권성동 영장 청구\t안미현 검사 외압 폭로 ...,0
3,"친문 홍영표, 문빠에 찍혔다…특검 합의에 문자폭탄 공격\t대표적인 친(親)문재인계인...",0
4,"北, 연일 南비난…韓美정상회담 전 경고성 메시지 발신\t南, 맥스선더·태영호 등 불...",0
...,...,...
1595,"단일팀 추진' 대한카누연맹, 데상트코리아와 5년 후원 협약\t[스포티비뉴스=조형애 ...",7
1596,"올림픽 성공 뒷이야기... 서울대, 16일 이희범 평창 조직위원장 초청 특강\t[O...",7
1597,21일 개막 호치민3쿠션월드컵에 149명 참가 확정\t韓선수 46명 참가…1차 예선...,7
1598,"스포츠안전재단, 대축전에 안전필요성 알려\t[스타뉴스 채준 기자]\n\n\n스포츠안...",7


In [6]:
# 데이터셋 분리

train=data.sample(frac=0.9,random_state=609)
test=data.drop(train.index)
print(f'train shape {train.shape}')
print(f'test shape {test.shape}')

train shape (1440, 2)
test shape (160, 2)


In [7]:
# 토큰화, 불용어 제거
from konlpy.tag import Okt

tokenizer=Okt()

traintoken=utils.getToken(train.text,tokenizer)
testtoken=utils.getToken(test.text,tokenizer)


'''
def getToken(textlist,tokenizer):
    """한글만 남기고 불용어 제거 및 토큰화"""
    text_to_token=[]
    for idx,text in enumerate(textlist):
        # 한글빼고 다지우기 [정규식]
        text=re.sub('[^ㄱ-ㅎ가-힣]+',' ',text)

        # 토큰으로 변환 [norm,stem << 참고] 
        tokens=tokenizer.morphs(text,norm=True)
        for token in tokens:
            # 불용어 목록에 있는 단어 토큰 제거
            if token in stop_words:
                tokens.remove(token)
        text_to_token.append(tokens)

    return text_to_token # 문장 토큰화
'''

'\ndef getToken(textlist,tokenizer):\n    """한글만 남기고 불용어 제거 및 토큰화"""\n    text_to_token=[]\n    for idx,text in enumerate(textlist):\n        # 한글빼고 다지우기 [정규식]\n        text=re.sub(\'[^ㄱ-ㅎ가-힣]+\',\' \',text)\n\n        # 토큰으로 변환 [norm,stem << 참고] \n        tokens=tokenizer.morphs(text,norm=True)\n        for token in tokens:\n            # 불용어 목록에 있는 단어 토큰 제거\n            if token in stop_words:\n                tokens.remove(token)\n        text_to_token.append(tokens)\n\n    return text_to_token # 문장 토큰화\n'

In [8]:
# 단어사전 생성 및 넘버링
vocab=utils.get_vocab(traintoken,n_vocab=10000)

token_to_idx={token:idx for idx,token in enumerate(vocab)}
idx_to_token={idx:token for idx,token in enumerate(vocab)}

print(token_to_idx)

'''
def get_vocab(text_to_token, n_vocab):
    """단어 사전 생성: n_vocab 갯수만큼 자주 사용된 단어들을 포함"""    
    # text_to_token 내의 모든 토큰을 하나의 리스트로 만듦
    tokenlist = [token for token in text_to_token]
    
    # 단어의 출현 빈도를 기록할 Counter 객체 생성
    counter = Counter()
    
    # 각 토큰 리스트에 대해 빈도수 업데이트
    for tokens in tokenlist:
        # Counter 객체에 토큰 리스트를 전달해 빈도수를 업데이트
        counter.update(tokens)
    
    # 단어 사전의 시작 부분에 패딩('<pad>')과 사전에 없는 단어('<oov>')를 추가
    vocab = ['<pad>', '<oov>']
    
    # 가장 자주 등장하는 n_vocab개의 단어를 vocab에 추가
    for token, count in counter.most_common(n_vocab):
        vocab.append(token)
    
    # 생성된 단어 사전을 반환
    return vocab
'''

{'<pad>': 0, '<oov>': 1, '했다': 2, '일': 3, '하는': 4, '있다': 5, '이': 6, '적': 7, '년': 8, '이다': 9, '할': 10, '말': 11, '전': 12, '된': 13, '월': 14, '으로': 15, '는': 16, '에': 17, '의': 18, '기자': 19, '한다': 20, '있는': 21, '을': 22, '명': 23, '한국': 24, '서울': 25, '은': 26, '위': 27, '지난': 28, '됐다': 29, '이라고': 30, '에는': 31, '날': 32, '한': 33, '가': 34, '북한': 35, '시간': 36, '중': 37, '밝혔다': 38, '대해': 39, '원': 40, '대한': 41, '된다': 42, '미국': 43, '성': 44, '를': 45, '당': 46, '대통령': 47, '것': 48, '통해': 49, '이번': 50, '분': 51, '중국': 52, '관련': 53, '때문': 54, '사람': 55, '경찰': 56, '될': 57, '라고': 58, '이후': 59, '장': 60, '정부': 61, '하지': 62, '보다': 63, '같은': 64, '대표': 65, '조사': 66, '사실': 67, '문제': 68, '후보': 69, '함께': 70, '대회': 71, '경우': 72, '상황': 73, '관계자': 74, '김': 75, '라며': 76, '점': 77, '지난해': 78, '이어': 79, '따르면': 80, '되는': 81, '혐의': 82, '진행': 83, '의원': 84, '최근': 85, '연': 86, '도': 87, '현재': 88, '확인': 89, '입니다': 90, '시장': 91, '자신': 92, '주장': 93, '달': 94, '하기': 95, '다': 96, '예정': 97, '결과': 98, '돼': 99, '하며': 100, '간': 101, '설명': 102,

'\ndef get_vocab(text_to_token, n_vocab):\n    """단어 사전 생성: n_vocab 갯수만큼 자주 사용된 단어들을 포함"""    \n    # text_to_token 내의 모든 토큰을 하나의 리스트로 만듦\n    tokenlist = [token for token in text_to_token]\n    \n    # 단어의 출현 빈도를 기록할 Counter 객체 생성\n    counter = Counter()\n    \n    # 각 토큰 리스트에 대해 빈도수 업데이트\n    for tokens in tokenlist:\n        # Counter 객체에 토큰 리스트를 전달해 빈도수를 업데이트\n        counter.update(tokens)\n    \n    # 단어 사전의 시작 부분에 패딩(\'<pad>\')과 사전에 없는 단어(\'<oov>\')를 추가\n    vocab = [\'<pad>\', \'<oov>\']\n    \n    # 가장 자주 등장하는 n_vocab개의 단어를 vocab에 추가\n    for token, count in counter.most_common(n_vocab):\n        vocab.append(token)\n    \n    # 생성된 단어 사전을 반환\n    return vocab\n'

In [9]:
trainnum=utils.padding_vectorize(traintoken,token_to_idx,pad_length=250)
testnum=utils.padding_vectorize(testtoken,token_to_idx,pad_length=250)

'''
def padding_vectorize(tokenlist, token_to_idx, pad_length):
    """ 토큰 리스트를 패딩 길이에 맞추고, 수치화 """
    
    # 수치화된 결과(인덱스 리스트)를 저장할 리스트
    idxlist = []
    
    # OOV (Out Of Vocabulary: 사전에 없는 단어) 토큰의 인덱스 값을 가져옴
    oov_num = token_to_idx['<oov>']
    
    # tokenlist에 있는 각 토큰 리스트에 대해 반복
    for tokens in tokenlist:
        # 토큰 길이가 pad_length보다 크거나 같으면 자름
        if len(tokens) >= pad_length:
            tokens = tokens[:pad_length]
        # 토큰 길이가 pad_length보다 짧으면 '<pad>'로 패딩
        else:
            tokens = tokens + (['<pad>'] * (pad_length - len(tokens)))
        
        # 각 토큰을 token_to_idx 사전에서 해당 인덱스로 변환
        # 사전에 없는 토큰은 OOV 토큰의 인덱스로 변환
        numbers = [token_to_idx.get(token, oov_num) for token in tokens]
        
        # 변환된 인덱스 리스트를 idxlist에 추가
        idxlist.append(numbers)
    
    # 결과 리스트를 numpy 배열로 변환하여 반환
    return np.array(idxlist)
'''

'\ndef padding_vectorize(tokenlist, token_to_idx, pad_length):\n    """ 토큰 리스트를 패딩 길이에 맞추고, 수치화 """\n    \n    # 수치화된 결과(인덱스 리스트)를 저장할 리스트\n    idxlist = []\n    \n    # OOV (Out Of Vocabulary: 사전에 없는 단어) 토큰의 인덱스 값을 가져옴\n    oov_num = token_to_idx[\'<oov>\']\n    \n    # tokenlist에 있는 각 토큰 리스트에 대해 반복\n    for tokens in tokenlist:\n        # 토큰 길이가 pad_length보다 크거나 같으면 자름\n        if len(tokens) >= pad_length:\n            tokens = tokens[:pad_length]\n        # 토큰 길이가 pad_length보다 짧으면 \'<pad>\'로 패딩\n        else:\n            tokens = tokens + ([\'<pad>\'] * (pad_length - len(tokens)))\n        \n        # 각 토큰을 token_to_idx 사전에서 해당 인덱스로 변환\n        # 사전에 없는 토큰은 OOV 토큰의 인덱스로 변환\n        numbers = [token_to_idx.get(token, oov_num) for token in tokens]\n        \n        # 변환된 인덱스 리스트를 idxlist에 추가\n        idxlist.append(numbers)\n    \n    # 결과 리스트를 numpy 배열로 변환하여 반환\n    return np.array(idxlist)\n'

In [10]:
import torch

train_text=torch.tensor(trainnum)
train_label=torch.FloatTensor(train.label.values)

test_text=torch.tensor(testnum)
test_label=torch.FloatTensor(test.label.values)


In [11]:
from torch.utils.data import TensorDataset,DataLoader

trainDS=TensorDataset(train_text,train_label)
testDS=TensorDataset(test_text,test_label)

trainDL=DataLoader(trainDS,batch_size=16)
testDL=DataLoader(testDS,batch_size=16)

In [12]:
model=utils.SentenceClassifier(n_vocab=len(token_to_idx),hidden_dim=64,embedding_dim=64,n_layers=4,feature_n=len(file_list))

import torch.optim as optim
import torch.nn as nn
from torchmetrics.classification import MulticlassF1Score
import torch.optim.lr_scheduler as lr_scheduler

optimizer=optim.Adam(model.parameters(),lr=0.001)
lossfun=nn.CrossEntropyLoss()
scorefun=MulticlassF1Score(num_classes=len(file_list))
scheduler=lr_scheduler.ReduceLROnPlateau(optimizer,patience=10,factor=0.3,mode='max')


In [13]:
EPOCH=100
TV=utils.Train_val(trainDL,testDL,model,optimizer,lossfun,scorefun)
train=TV.train(EPOCH,scheduler,1)

[1/100]
train loss 2.0835296074549357, train score 0.05955217596557405
val loss 2.0206985473632812, val score 0.46666666865348816
scheduler.num_bad_epochs 0/10
[2/100]
train loss 2.0320912149217394, train score 0.08243196287916767
val loss 1.3969250917434692, val score 0.46666666865348816
scheduler.num_bad_epochs 1/10
[3/100]
train loss 1.9684301336606345, train score 0.09943798132654694
val loss 1.4806638956069946, val score 0.4838709533214569
scheduler.num_bad_epochs 0/10
[4/100]
train loss 1.9029373619291516, train score 0.11136172525584698
val loss 1.6722288131713867, val score 0.29885056614875793
scheduler.num_bad_epochs 1/10
[5/100]
train loss 1.8358108056916131, train score 0.14215162448171112
val loss 1.5682439804077148, val score 0.31111112236976624
scheduler.num_bad_epochs 2/10
[6/100]
train loss 1.7948421279589335, train score 0.1730169607533349
val loss 1.1389410495758057, val score 0.4838709533214569
scheduler.num_bad_epochs 3/10
[7/100]
train loss 1.6328960365719265, trai

[14/100]
train loss 0.29559781464437646, train score 0.874068041642507   
val loss 0.028206512331962585, val score 1.0   

끝!!!!!!!!!!!!!!!!!!!!!!!!!!

'2.3.0'